In [1]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [6]:
#import data
bed_types = ['Studio', '1Bedroom', '3Bedroom', '4Bedroom', '5BedroomOrMore']
dataset = [None]*len(bed_types)
for index, term in enumerate(bed_types):
    print(term)
    temp_df=pd.read_csv('data/Zip_MedianRentalPrice_{0}.csv'.format(term))
    temp_df = temp_df[temp_df['City'] == 'Seattle']
    dataset[index] = temp_df


Studio
     RegionName     City State    Metro CountyName  SizeRank  2010-03  \
70        98103  Seattle    WA  Seattle       King        71      NaN   
156       98122  Seattle    WA  Seattle       King       157      NaN   
251       98109  Seattle    WA  Seattle       King       252      NaN   
268       98105  Seattle    WA  Seattle       King       269      NaN   
327       98102  Seattle    WA  Seattle       King       328      NaN   
445       98107  Seattle    WA  Seattle       King       446      NaN   
513       98121  Seattle    WA  Seattle       King       514      NaN   
663       98101  Seattle    WA  Seattle       King       664      NaN   
733       98104  Seattle    WA  Seattle       King       734      NaN   

     2010-04  2010-05  2010-06   ...     2017-05  2017-06  2017-07  2017-08  \
70       NaN      NaN      NaN   ...      1595.0   1660.0   1395.0   1395.0   
156      NaN      NaN      NaN   ...      1495.0   1627.5   1627.5   1612.5   
251      NaN      NaN    